In [ ]:
import numpy as np
from datascience import *
from math import *
import pandas as pd

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import seaborn as sns

## Coding Review: Creating a confidence interval

How accurate is the confidence interval process? Let's find out! We have some **population-level** (let's assume) data of city employee salaries in Oakland in 2018. (Source: Transparent California)

In reality, you would not often have access to information of this level. Let's assume that we only have access to samples from this population, so let's see if the confidence interval process works!

We're also going to do a hypothesis test (for practice) with a confidence interval here. 

In 2019, the average total compensation -- salary and benefits -- for a full-time employee in California cities was approximately $ 174,000 (according to the California Globe). Is it the same for Oakland? 

H0: The average total pay and benefits for a city employee in Oakland is $174,000.

Ha: The average total pay and benefits for a city employee in Oakland is **not** $174,000.

Note: Does that number seem kind of high? In this analysis, we include both firefighters and police officers, who tend to have higher salaries and wages due to their unions, and therefore skew the data.

In [ ]:
# Run this cell.
oakland = Table().read_table("oakland-2018.csv").where("Status", "FT") #Full time workers only
pop_mean = np.mean(oakland.column("Total Pay & Benefits"))
oakland.show(5)
print("The population mean is: $", np.round(pop_mean, 2)) # For readability: don't round the mean in your own code!
oakland.hist("Total Pay & Benefits")
plots.axvline(x = pop_mean, y = 0, color = "red");

In [ ]:
# Q1: I wrote a function for you that does the bootstrapping process.
# Let's practice understanding what code does. Assume "tbl" is in the same format 
# as the oakland table. 

def one_full_bootstrap(tbl, label, n, reps, c):
    # Step 1) Type your comment here. 
    one_sample = tbl.sample(n, with_replacement = False) #...
    bootstrap_stats = np.array([]) # ...
    for i in range(reps): #... 
        # Step 2: ...
        resample = one_sample.sample() # ...
        # Step 3: ...
        resample_stat = np.mean(resample.column(label)) # ...
        bootstrap_stats = np.append(bootstrap_stats, resample_stat) # ...
    # Step 4: ...
    lower_bound = percentile((100 - c)/2, bootstrap_stats) # ...
    upper_bound = percentile(100 - ((100-c)/2), bootstrap_stats) # ...
    plots.hist(bootstrap_stats, density = True, bins = np.arange(min(bootstrap_stats), max(bootstrap_stats), 5000)) # ...
    plots.axvline(lower_bound, color = "gold") # ...
    plots.axvline(upper_bound, color = "gold") # ...
    plots.axvline(pop_mean, color = "red")  # ...
    return np.array([lower_bound, upper_bound]) # ...

In [ ]:
# Just run this cell!
one_interval = one_full_bootstrap(oakland, "Total Pay & Benefits", 50, 1000, 90)
print("One random interval has a lower bound of ", str(one_interval.item(0)), " and an upper bound of ", str(one_interval.item(1)))

In [ ]:
# Q2: if we ran the code above 1000 times, how many intervals would capture the pop mean on average?
# Type your guess in estimated_correct below.
num_intervals = 1000
estimated_correct = ...

In [ ]:
# Q3: Do we reject or fail to reject our null hypothesis?  Set reject_null to True if we reject
# and False to if we fail to reject. 
# Use the interval, one_interval, from above, to make this conclusion. 

reject_null = ???

#Q5: What is our significance level for this test? 

significance_level = ???

## Center & Spread

https://docs.scipy.org/doc/scipy/reference/stats.html

In [ ]:
## Out of scope of this class (but mentioned in the textbook!):
# Unlike R, Python is not designed for statistical analysis
# BUT - we can use libraries for statistical analysis, like numpy and scipy

from scipy import stats # Import the stats module from scipy; call with stats.function(...)
# Lot of cool stats functions! 

# For example: Mayor Schaaf made $314,400 in Total Pay & Benefits. What percentile of employees was she at?
# Our function in scipy.stats: percentileofscore(array, value)
pay_benefits = oakland.column("Total Pay & Benefits")
...

In [ ]:
## Measures of center in Python
np.mean(...)
np.median(...)
percentile(..., ...)  # Note: percentile is a datascience library function; np.percentile(array, percent) is in numpy
stats.mode(..., axis = 0)

In [ ]:
## Measures of spread in Python
np.std(...)
np.ptp(...) # range
stats.iqr(...) # interquartile range: 75th quartile - 25th quartile

In [ ]:
# Other visual representations of a quantitative distribution above:
sns.boxplot(y = pay_benefits);

sns.displot(pay_benefits, kde = True, rug = True, aspect = 3); 

In [ ]:
## Recall: Variance = average of the squared differences from the mean
# short-answer: use np.var
print("Numpy says the variance is: ", np.var(pay_benefits))

# Now, do it yourself: write a function that calculates the variance of an array
def your_var(arr):
    ...

your_var(pay_benefits)

In [ ]:
# Write a function that calculates the standard deviation!
# Don't forget about abstraction (use the your_var function)

print("Numpy says the standard deviation is: " + str(np.std(pay_benefits)))

def your_std(arr):
   ...

your_std(pay_benefits)

## Normality

The normal curve is a particular bell-shaped distribution, where 68% of data is +-1 SD from the mean, 95% +- 2 SD from the mean, and 99% +- 3 SD from the mean. Contrast this with all distributions which follow Chebyshev's bounds: at least 0% for 1 SD, 75% for 2 SDs, and 88% for 3 SDs. Furthermore, the normal curve is nice in that the SD is the distance between the mean and the point of inflection on the curve on the left or right. 

Note that, because we are using a simulation to approximate some values for this exercise, we may NOT have the exact results we're looking for.

In [ ]:
## Before we begin, we assume the Central Limit Theorem holds
# in this case: we have a large random sample using the sample mean or sample sum
# Without replacement OK because we have a small sample in relation to pop
# Let's look at the data!
# We're going to use Total Pay in this specific case, NOT pay and benefits

oakland.hist("Total Pay", bins = np.arange(0, 400000, 50000)) ## The population distribution
plots.title("Population");

our_sample = oakland.sample(30, with_replacement = False) # notice sample size of 30
our_sample.hist("Total Pay", bins = np.arange(0, 400000, 50000)) ## Our sample distribution
plots.title("Sample");

In [ ]:
# Q: Now, let's create a distribution of sample means
# Sample withOUT replacement, n = 30, from oakland, and calculate the means

sample_means = make_array()
trials = 1000

for ...

means_tbl = Table().with_column("Resampled Means", sample_means)
means_tbl.hist()

In [ ]:
## Looks good! Q: Is it approx. normal as stated by the CLT? Let's check using a "back of the envelope" test
# i.e. check if the 68-95-99.7 rule works here

sd = ...
avg = ...


# Checking if the rule fits
left = sd - avg
right = sd + avg

sum((left <= sample_means) & (sample_means <= right)) / len(sample_means)  
# & refers to a bit-wise comparison, when we compare 2 arrays

In [ ]:
# We just did the traditional statistical approach above
# What if we used a bootstrap, resampling WITH replacement?
print("Our SD with a sample size of 30 was: " + str(sd))

new_sample_size = 30 # One last note: what happens to the variation when we change the sample size?
our_new_sample = oakland.sample(new_sample_size, with_replacement = False)
new_sample_means = make_array()

# The bootstrap approach:
for i in np.arange(1000):
    resample = our_new_sample.sample()
    resamp_mean = np.mean(resample.column("Total Pay"))
    new_sample_means = np.append(new_sample_means, resamp_mean)
    
bootstrap_means_tbl = Table().with_column("Resampled Means", sample_means)
bootstrap_means_tbl.hist()
    
# What's our SD now?
print("Our SD with a sample size of " + str(new_sample_size) + " was: " + str(np.std(new_sample_means)))

In [ ]:
## Why does this occur?

# Conceptual explanation:


# Mathematical explanation: 



## Correlation: What is the relationship between 2 variables?

In [ ]:
# Does one's base salary correlate with the amount of benefits they receive?
# Let's check graphically:

oakland.scatter("Base Pay", "Benefits")

In [ ]:
## It would appear so! But we need to prove it quantitatively.
# First, notice the ranges of each of the axes. 
np.ptp(...) # calculates max - min

In [ ]:
## A potential issue is any calculation we make
# may be more reflective of the vast differences in the ranges,
# rather than an actual correlation. So first, we need to standardize!

def standard_units(arr):
    return ...

In [ ]:
standard_oak = ...

standard_oak.scatter(0, 1)

# What do you notice?

In [ ]:
## Now, let's quantify the relationship between the 2 variables
# We will use the correlation coefficient, r

def r(tbl):
    """Given a 2 column table of x and y in STANDARD UNITS, calculate the correlation coefficient"""
   ...

r(standard_oak)

In [ ]:
## What should we conclude from above?

# as another tip: another way we can calculate R (without all of this code):
stats.pearsonr(oakland.column("Base Pay"), oakland.column("Benefits"))

# SciPy reports 2 values: the correlation coefficient and the p-value (w/ the null that x & y are uncorrelated)

In [ ]:
# What we will do starting next week:
# How do we calculate this line mathematically? 

oak_df = oakland.to_df()

sns.lmplot(x = "Base Pay", y= "Benefits", data = oak_df, scatter_kws={"s": 4}, line_kws = {"color": "gold", "linewidth": 3}, aspect = 1.5);
